In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess, covariance_lib
import matplotlib.pyplot as plt
import pickle 

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

Names in str: 'Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)'

In [2]:
strategy3 = True

Nt =110

max_manual_num = 5 
min_interval_num = 10

In [3]:
if strategy3:
    static_ind = [0,1,2,3,4,5,6,7,8,9,10]
    dynamic_ind = [3,7,11,12,13]
    all_ind = static_ind+dynamic_ind

else:
    static_ind = [0,1,2,4,5,6,8,9,10]
    dynamic_ind = [3,7,11,12,13]

num_total = len(all_ind)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 3, 7, 11, 12, 13]


In [4]:
all_names_strategy3 = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T',  'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', # all static
            'Ads.gas_outlet.z("CO2").static', 'Des.gas_outlet.z("CO2").static', # static z 
            'Ads.gas_outlet.z("CO2").dynamic', 'Des.gas_outlet.z("CO2").dynamic', # dynamic z 
            'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

In [5]:
if strategy3:
    static_cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         500, #ads.gas_outlet.T (2)
        7000,
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         500, #des.gas_outlet.T (6)
        7000,
         1000, #ads.T19 (8)
         1000, #ads.T23 (9)
         1000 #ads.T28 (10)
       ]
else:
    static_cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         500, #ads.gas_outlet.T (2)
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         500, #des.gas_outlet.T (6)
         1000, #ads.T19 (8)
         1000, #ads.T23 (9)
         1000 #ads.T28 (10)
       ]

static_cost.extend([100, 100, 500, 500, 500])

dynamic_cost = [0]*len(static_ind)
dynamic_cost.extend([100]*len(dynamic_ind))

max_manual = [max_manual_num]*num_total
min_time_interval = [min_interval_num]*num_total

In [6]:
measure_info = pd.DataFrame({
    "name": all_names_strategy3,
    "Q_index": all_ind,
        "static_cost": static_cost,
    "dynamic_cost": dynamic_cost,
    "min_time_interval": min_time_interval, 
    "max_manual_number": max_manual
})

print(measure_info)

                               name  Q_index  static_cost  dynamic_cost  \
0                   Ads.gas_inlet.F        0         1000             0   
1                  Ads.gas_outlet.F        1         1000             0   
2                  Ads.gas_outlet.T        2          500             0   
3                   Des.gas_inlet.F        3         7000             0   
4                  Des.gas_outlet.F        4         1000             0   
5                  Des.gas_outlet.T        5         1000             0   
6              Ads.T_g.Value(19,10)        6          500             0   
7              Ads.T_g.Value(23,10)        7         7000             0   
8              Ads.T_g.Value(28,10)        8         1000             0   
9    Ads.gas_outlet.z("CO2").static        9         1000             0   
10   Des.gas_outlet.z("CO2").static       10         1000             0   
11  Ads.gas_outlet.z("CO2").dynamic        3          100           100   
12  Des.gas_outlet.z("CO2

In [7]:
dataObject = DataProcess()
dataObject.read_jaco('./RotaryBed/Q'+str(Nt)+'_scale.csv')
Q = dataObject.get_Q_list(measure_info, Nt)


(16, 110, 5)


In [8]:
calculator = MeasurementOptimizer(Q, measure_info, error_opt=covariance_lib.mode0, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

(314721, 5, 5)


### Debug functions: check each unit FIM

In [9]:
def check(fim_set):
    for i in range(num_total):
        for j in range(num_total):
            fim = fim_set[i*num_total+j]
            if np.any(fim):
                logdet = np.log(np.linalg.det(fim))
                if logdet < 0:
                    print("unit FIM with small Det: ", i, j, ', logdet:', logdet)
                if max(np.linalg.eigvals(fim))/min(np.linalg.eigvals(fim)) > 1000:
                    print("unit FIM with big condition number:", i, j, ", eigvals:", np.linalg.eigvals(fim))

In [10]:
def print_fim(t):
    fim = calculator.fim_collection[t*num_total+t]
    print(fim)
    print(np.linalg.eigvals(fim))

total_fim = np.zeros((5,5))
for i in range(314721):
    total_fim += calculator.fim_collection[i]
print(total_fim)

[[ 5.68675822e+03  3.40446058e+00 -4.12615577e+03  1.12130355e+04
  -6.89977591e+02]
 [ 3.40446058e+00  5.76986520e+00  7.18420470e+00  6.82874325e+01
  -1.97005732e+01]
 [-4.12615577e+03  7.18420470e+00  3.34118481e+03 -8.54865956e+03
   1.50792220e+02]
 [ 1.12130355e+04  6.82874325e+01 -8.54865956e+03  2.60398825e+04
  -1.23576756e+03]
 [-6.89977591e+02 -1.97005732e+01  1.50792220e+02 -1.23576756e+03
   6.81552017e+02]]


### Initialize

In [11]:
num_static = len(static_ind)
num_dynamic  = len(dynamic_ind)
num_total = num_static + num_dynamic*Nt

In [12]:
with open('Mar20_A_110_20000_strategy3_mip', 'rb') as f:
    x = pickle.load(f)
    
print(x)

init_cov_y = x.copy()

for i in range(11):
    init_cov_y[i,i] = 1
    
# initialize all decisions
#init_cov_y = np.ones(x.shape) 

[[1. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
#with open('strategy3_all_covy', 'rb') as f:
with open('FIM_strategy3_A_110_20000', 'rb') as f:
    fim_prior = pickle.load(f)
    
print(fim_prior)

[[ 5.54781941e+03 -2.79896951e+00 -4.14297808e+03  1.09902329e+04
  -4.94072165e+02]
 [-2.79896951e+00  5.20826025e+00  7.60848042e+00  5.06824563e+01
  -1.04261168e+01]
 [-4.14297808e+03  7.60848042e+00  3.32483923e+03 -8.52165366e+03
   1.68122219e+02]
 [ 1.09902329e+04  5.06824563e+01 -8.52165366e+03  2.52864273e+04
  -8.96811655e+02]
 [-4.94072165e+02 -1.04261168e+01  1.68122219e+02 -8.96811655e+02
   3.67315096e+02]]


In [15]:

mip_option = False
objective = "D"
sparse_opt = True
fix_opt = False

manual_num = 20
budget_opt = 27700

num_dynamic_time = np.linspace(2,220,Nt)

dynamic_time_dict = {}
for i, tim in enumerate(num_dynamic_time):
    dynamic_time_dict[i] = tim 

In [16]:
mod = calculator.continuous_optimization(mixed_integer=mip_option, 
                      obj=objective, 
                    fix=fix_opt, 
                    sparse=sparse_opt, 
                    num_dynamic_t_name = num_dynamic_time, 
                    manual_number = manual_num, 
                    budget=budget_opt,
                    init_cov_y= init_cov_y, 
                    initial_fim = fim_prior)

mod = calculator.solve(mod, mip_option=mip_option, objective = objective)

{'ele_0_0': 5547.819413862037, 'ele_0_1': -2.7989695079930907, 'ele_0_2': -4142.978078958834, 'ele_0_3': 10990.232885489535, 'ele_0_4': -494.07216522961767, 'ele_1_1': 5.208260254418963, 'ele_1_2': 7.608480416183696, 'ele_1_3': 50.68245631431674, 'ele_1_4': -10.426116776757201, 'ele_2_2': 3324.8392276658083, 'ele_2_3': -8521.653655909617, 'ele_2_4': 168.12221904300958, 'ele_3_3': 25286.42731937437, 'ele_3_4': -896.8116552992137, 'ele_4_4': 367.3150958511481}
initialize with: {'ele_0_0': 5547.819413862037, 'ele_0_1': -2.7989695079930907, 'ele_0_2': -4142.978078958834, 'ele_0_3': 10990.232885489535, 'ele_0_4': -494.07216522961767, 'ele_1_1': 5.208260254418963, 'ele_1_2': 7.608480416183696, 'ele_1_3': 50.68245631431674, 'ele_1_4': -10.426116776757201, 'ele_2_2': 3324.8392276658083, 'ele_2_3': -8521.653655909617, 'ele_2_4': 168.12221904300958, 'ele_3_3': 25286.42731937437, 'ele_3_4': -896.8116552992137, 'ele_4_4': 367.3150958511481}
initialized
initialized
initialized
initialized
initializ


 Consider M =
 [[ 5.46930705e+03 -2.72342363e+00 -4.08298286e+03  1.08324567e+04
  -4.88486513e+02]
 [-2.72342363e+00  5.14906086e+00  7.51822046e+00  5.01057380e+01
  -1.03151881e+01]
 [-4.08298286e+03  7.51822046e+00  3.27742831e+03 -8.39952467e+03
   1.65104118e+02]
 [ 1.08324567e+04  5.01057380e+01 -8.39952467e+03  2.49274125e+04
  -8.84651792e+02]
 [-4.88486513e+02 -1.03151881e+01  1.65104118e+02 -8.84651792e+02
   3.64703176e+02]]
[5469.307052606059, -2.7234236331775676, -4082.9828560525516, 10832.456746185575, -488.4865134302867, 5.149060857787661, 7.5182204646396515, 50.10573804427166, -10.31518809981573, 3277.428308142431, -8399.52467192802, 165.10411837939054, 24927.412456322716, -884.6517917632074, 364.703175582974]
   logdet =  28.27943465618685 

Eigvals: [3.27497425e+04 8.13314372e+02 4.32112089e+02 4.51509141e+01
 3.68021442e+00]

 Consider M =
 [[ 5.46823669e+03 -2.72329563e+00 -4.08217216e+03  1.08302832e+04
  -4.88392922e+02]
 [-2.72329563e+00  5.14844176e+00  7.5171


 Consider M =
 [[ 5.45932329e+03 -2.75016074e+00 -4.07598020e+03  1.08122868e+04
  -4.86771340e+02]
 [-2.75016074e+00  5.14051285e+00  7.52093528e+00  4.99060257e+01
  -1.02455352e+01]
 [-4.07598020e+03  7.52093528e+00  3.27181271e+03 -8.38490790e+03
   1.64708574e+02]
 [ 1.08122868e+04  4.99060257e+01 -8.38490790e+03  2.48786366e+04
  -8.80787539e+02]
 [-4.86771340e+02 -1.02455352e+01  1.64708574e+02 -8.80787539e+02
   3.62853366e+02]]
[5459.323290828241, -2.750160744434096, -4075.980203143716, 10812.286774164879, -486.771339792321, 5.14051285486464, 7.520935277144455, 49.90602569378288, -10.245535183045952, 3271.812705007464, -8384.907901695022, 164.7085743681753, 24878.63661222969, -880.787538684246, 362.85336575422815]
   logdet =  28.2661401014533 

Eigvals: [3.26875906e+04 8.11365284e+02 4.30125907e+02 4.50090226e+01
 3.67565005e+00]

 Consider M =
 [[ 5.45471428e+03 -2.84075633e+00 -4.07379552e+03  1.08033962e+04
  -4.83671537e+02]
 [-2.84075633e+00  5.12921219e+00  7.53141238e


 Consider M =
 [[ 5.54326710e+03 -2.92463725e+00 -4.14122623e+03  1.09797569e+04
  -4.90629399e+02]
 [-2.92463725e+00  5.18158657e+00  7.66051690e+00  4.99964366e+01
  -1.02128561e+01]
 [-4.14122623e+03  7.66051690e+00  3.32323712e+03 -8.51675692e+03
   1.67682471e+02]
 [ 1.09797569e+04  4.99964366e+01 -8.51675692e+03  2.52459072e+04
  -8.87605994e+02]
 [-4.90629399e+02 -1.02128561e+01  1.67682471e+02 -8.87605994e+02
   3.62638139e+02]]
[5543.267100133521, -2.9246372466017307, -4141.226229538224, 10979.756920929074, -490.6293986408089, 5.181586567123214, 7.660516898056841, 49.996436619760104, -10.212856139199832, 3323.237117473936, -8516.756918470366, 167.68247112523437, 25245.90718427822, -887.6059936801132, 362.6381394594637]
   logdet =  28.30595124122551 

Eigvals: [3.31812965e+04 8.19901542e+02 4.30084678e+02 4.52381536e+01
 3.71026161e+00]

 Consider M =
 [[ 5.54274201e+03 -2.92558840e+00 -4.14071810e+03  1.09779877e+04
  -4.90830795e+02]
 [-2.92558840e+00  5.18349046e+00  7.668


 Consider M =
 [[ 5.53733477e+03 -3.12896725e+00 -4.13882378e+03  1.09642112e+04
  -4.85894910e+02]
 [-3.12896725e+00  5.16235952e+00  7.71464959e+00  4.92429478e+01
  -9.89240547e+00]
 [-4.13882378e+03  7.71464959e+00  3.32146323e+03 -8.51135619e+03
   1.66943920e+02]
 [ 1.09642112e+04  4.92429478e+01 -8.51135619e+03  2.51974930e+04
  -8.73461904e+02]
 [-4.85894910e+02 -9.89240547e+00  1.66943920e+02 -8.73461904e+02
   3.55934607e+02]]
[5537.3347652299535, -3.128967251893426, -4138.8237758981795, 10964.21117738501, -485.8949097743727, 5.162359518523386, 7.714649585117774, 49.24294782967816, -9.892405467778842, 3321.463233796678, -8511.35618910008, 166.94391977711285, 25197.493046722062, -873.4619037531543, 355.9346073734293]
   logdet =  28.275450730313345 

Eigvals: [3.31288310e+04 8.16545328e+02 4.23415531e+02 4.48914829e+01
 3.70471737e+00]

 Consider M =
 [[ 5.53674212e+03 -3.14306427e+00 -4.13856823e+03  1.09629259e+04
  -4.85492808e+02]
 [-3.14306427e+00  5.16035540e+00  7.7173

...


 Consider M =
 [[ 5.54933028e+03 -2.71702021e+00 -4.14161418e+03  1.09850390e+04
  -4.96876805e+02]
 [-2.71702021e+00  5.20125672e+00  7.73977408e+00  5.00555746e+01
  -1.04139794e+01]
 [-4.14161418e+03  7.73977408e+00  3.32442868e+03 -8.51840879e+03
   1.66976669e+02]
 [ 1.09850390e+04  5.00555746e+01 -8.51840879e+03  2.52411980e+04
  -8.92720303e+02]
 [-4.96876805e+02 -1.04139794e+01  1.66976669e+02 -8.92720303e+02
   3.72872447e+02]]
[5549.330281983857, -2.7170202100539336, -4141.61418358206, 10985.03896976559, -496.87680456544246, 5.201256720975953, 7.739774082271295, 50.05557463949321, -10.413979436440775, 3324.4286771504276, -8518.408793780913, 166.976669403585, 25241.197986946117, -892.7203026766364, 372.8724467356716]
   logdet =  28.348162941368514 

Eigvals: [3.31837506e+04 8.17888625e+02 4.41736239e+02 4.59353548e+01
 3.71981202e+00]

 Consider M =
 [[ 5.54953829e+03 -2.71171615e+00 -4.14162637e+03  1.09852938e+04
  -4.97081600e+02]
 [-2.71171615e+00  5.20221503e+00  7.7371


 Consider M =
 [[ 5.55044448e+03 -2.68761552e+00 -4.14161632e+03  1.09863111e+04
  -4.97920871e+02]
 [-2.68761552e+00  5.20974498e+00  7.70795789e+00  5.02151497e+01
  -1.04510882e+01]
 [-4.14161632e+03  7.70795789e+00  3.32494128e+03 -8.51962360e+03
   1.66895758e+02]
 [ 1.09863111e+04  5.02151497e+01 -8.51962360e+03  2.52455348e+04
  -8.94085418e+02]
 [-4.97920871e+02 -1.04510882e+01  1.66895758e+02 -8.94085418e+02
   3.74405171e+02]]
[5550.444484973098, -2.6876155169695357, -4141.616324573817, 10986.311145241089, -497.92087103149777, 5.2097449757247905, 7.707957892525612, 50.21514965772979, -10.451088223463584, 3324.9412778300716, -8519.623599066375, 166.89575813904196, 25245.534849372198, -894.0854184936297, 374.40517097223545]
   logdet =  28.36019818694401 

Eigvals: [3.31888434e+04 8.18162479e+02 4.43596956e+02 4.62075086e+01
 3.72515880e+00]

 Consider M =
 [[ 5.55062487e+03 -2.68305519e+00 -4.14160598e+03  1.09865230e+04
  -4.98116139e+02]
 [-2.68305519e+00  5.21146350e+00  7


 Consider M =
 [[ 5.55156147e+03 -2.65092956e+00 -4.14151684e+03  1.09874051e+04
  -4.99179154e+02]
 [-2.65092956e+00  5.21458693e+00  7.69903627e+00  5.03038130e+01
  -1.04975464e+01]
 [-4.14151684e+03  7.69903627e+00  3.32528908e+03 -8.52027237e+03
   1.66704174e+02]
 [ 1.09874051e+04  5.03038130e+01 -8.52027237e+03  2.52478141e+04
  -8.95428959e+02]
 [-4.99179154e+02 -1.04975464e+01  1.66704174e+02 -8.95428959e+02
   3.76260540e+02]]
[5551.56147265, -2.6509295637396355, -4141.516841703622, 10987.4050865336, -499.1791538022189, 5.214586928062967, 7.699036271407106, 50.303812958009225, -10.497546392416904, 3325.2890756580646, -8520.272374486876, 166.70417356768812, 25247.814125847424, -895.4289592441424, 376.260540397927]
   logdet =  28.370040429282966 

Eigvals: [3.31919287e+04 8.18222434e+02 4.45873230e+02 4.63878144e+01
 3.72763026e+00]

 Consider M =
 [[ 5.55161853e+03 -2.64919206e+00 -4.14150047e+03  1.09874305e+04
  -4.99245973e+02]
 [-2.64919206e+00  5.21482525e+00  7.6989676


 Consider M =
 [[ 5.55226400e+03 -2.63110065e+00 -4.14135402e+03  1.09879744e+04
  -5.00044606e+02]
 [-2.63110065e+00  5.21866370e+00  7.68637056e+00  5.03644189e+01
  -1.05201010e+01]
 [-4.14135402e+03  7.68637056e+00  3.32558802e+03 -8.52072033e+03
   1.66473901e+02]
 [ 1.09879744e+04  5.03644189e+01 -8.52072033e+03  2.52489270e+04
  -8.96151798e+02]
 [-5.00044606e+02 -1.05201010e+01  1.66473901e+02 -8.96151798e+02
   3.77637304e+02]]
[5552.2640000687625, -2.631100654427185, -4141.354021188618, 10987.974374726611, -500.04460602099203, 5.218663697625717, 7.6863705561030065, 50.364418904149304, -10.520101014494173, 3325.588023814356, -8520.720334161177, 166.4739008677923, 25248.92700171226, -896.1517976061722, 377.63730387724047]
   logdet =  28.37819820508566 

Eigvals: [3.31935445e+04 8.18196563e+02 4.47616429e+02 4.65468752e+01
 3.73067191e+00]

 Consider M =
 [[ 5.55229143e+03 -2.62976885e+00 -4.14135159e+03  1.09879802e+04
  -5.00072296e+02]
 [-2.62976885e+00  5.21887298e+00  7.6


 Consider M =
 [[ 5.55258480e+03 -2.62000958e+00 -4.14131204e+03  1.09879568e+04
  -5.00363945e+02]
 [-2.62000958e+00  5.22027516e+00  7.68720839e+00  5.03870912e+01
  -1.05219266e+01]
 [-4.14131204e+03  7.68720839e+00  3.32562722e+03 -8.52086277e+03
   1.66402819e+02]
 [ 1.09879568e+04  5.03870912e+01 -8.52086277e+03  2.52492091e+04
  -8.96252640e+02]
 [-5.00363945e+02 -1.05219266e+01  1.66402819e+02 -8.96252640e+02
   3.78290172e+02]]
[5552.584799581312, -2.6200095800449814, -4141.312043738898, 10987.956791948272, -500.36394504067556, 5.220275158896256, 7.687208390710865, 50.387091184561655, -10.521926579753716, 3325.6272155662605, -8520.862772911776, 166.40281852374892, 25249.209069380096, -896.2526397699313, 378.2901716577053]
   logdet =  28.381818642294437 

Eigvals: [3.31938755e+04 8.18356879e+02 4.48349046e+02 4.66185016e+01
 3.73157319e+00]

 Consider M =
 [[ 5.55259779e+03 -2.61977524e+00 -4.14130974e+03  1.09879462e+04
  -5.00374848e+02]
 [-2.61977524e+00  5.22031918e+00  7

In [17]:
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(i,5):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))
print(np.linalg.eigvals(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

[[ 5.55260011e+03 -2.61972445e+00 -4.14130933e+03  1.09879444e+04
  -5.00376769e+02]
 [-2.61972445e+00  5.22032483e+00  7.68731822e+00  5.03877570e+01
  -1.05219552e+01]
 [-4.14130933e+03  7.68731822e+00  3.32562801e+03 -8.52086707e+03
   1.66399892e+02]
 [ 1.09879444e+04  5.03877570e+01 -8.52086707e+03  2.52492460e+04
  -8.96253626e+02]
 [-5.00376769e+02 -1.05219552e+01  1.66399892e+02 -8.96253626e+02
   3.78315460e+02]]
trace: 34511.00993581643
det: 2119056400016.21
[3.31938994e+04 8.18378909e+02 4.48378213e+02 4.66218092e+01
 3.73160950e+00]
Pyomo OF: 28.381992011863456
Log_det: 28.38199201119661


In [18]:
ans_y, sol_y = calculator.extract_solutions(mod)

Ads.gas_inlet.F :  1.0
Ads.gas_outlet.F :  1.0
Ads.gas_outlet.T :  1.0
Des.gas_inlet.F :  1.0
Des.gas_outlet.F :  1.0
Des.gas_outlet.T :  1.0
Ads.T_g.Value(19,10) :  1.0
Ads.T_g.Value(23,10) :  1.0
Ads.T_g.Value(28,10) :  1.0
Ads.gas_outlet.z("CO2").static :  1.0
Des.gas_outlet.z("CO2").static :  1.0
Ads.gas_outlet.z("CO2").dynamic
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Des.gas_outlet.z("CO2").dynamic
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0.

In [ ]:
print(ans_y)

In [22]:
# verify cost 
#real_cost = 0 
#for i in range(num_total):
#    real_cost += ans_y[i,i]*cost[i]
    
#dynamic_inst = []
#for j in range(num_static, num_dynamic+num_static):
#    dynamic_inst.append(pyo.value(mod.if_install_dynamic[j]))
#    for t in range(Nt):
#        if sol_y[j,t]>0.99:
#            dynamic_inst[j] = 1 
            
#real_cost += sum(dynamic_inst[i]*install_cost_dynamic[i] for i in range(num_dynamic))
#print(dynamic_inst)
#print('Real cost:', real_cost)
print('pyomo calculated cost:', pyo.value(mod.cost))

pyomo calculated cost: 25699.998773712417


In [ ]:
count = 0 
for i in range(561):
    for j in range(i,561):
        #if pyo.value(mod.cov_y[i,j])<0.96:
        print(pyo.value(mod.cov_y[i,j]))
            #count += 1 
            
print(count)

In [23]:
store = True

if store:
    file = open('Mar31_D_110_27700', 'wb')

    pickle.dump(ans_y, file)

    file.close()
    
    file2 = open('Mar31_FIM_D_110_27700', 'wb')

    pickle.dump(fim_result, file2)

    file2.close()